In [74]:
import requests
import pandas as pd
import json

# Movie ids coming from http://files.tmdb.org/p/exports/movie_ids_10_20_2020.json.gz
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

%matplotlib inline

try:
    fp = open('api_key.txt')
    api_key = fp.readline()
finally:
    fp.close()

In [75]:
tmp = pd.read_json("movie_ids_10_20_2020.json", lines=True)
all_movies = tmp.sort_values('popularity', ascending=False).reset_index(drop=True).head(25000).copy()
#del(tmp)

In [76]:
test = tmp.sort_values('popularity', ascending=False).reset_index(drop=True)
test[test.original_title=='The Love Guru']

,adult,id,original_title,popularity,video
10208,False,12177,The Love Guru,10.062,False


In [ ]:
# Getting Full info for all these movies
info_list = []

for m_id in tqdm.tqdm(list(all_movies.id)):
    r = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(m_id, api_key))
    info_list.append(json.loads(r.content.decode(encoding="ascii", errors="ignore")))

 42%|███████████████████████████████▋                                            | 10414/25000 [33:05<52:43,  4.61it/s]

In [ ]:
# Getting Cast List for all these movies
credit_list = []

for m_id in tqdm.tqdm(list(all_movies.id)):
    r = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}'.format(m_id, api_key))
    credit_list.append(json.loads(r.content.decode(encoding="ascii", errors="ignore")))

In [ ]:
# So we can come back and get the names of all these actors later if we need to. Really ids are totally fine for this

In [ ]:
movie_records = []
movie_features = ['adult','id','popularity','original_title','original_langue',
                  'release_date','revenue', 'vote_average', 'vote_count', 'budget']

for inf in info_list:
    try:
        movie_records.append([inf.get(key) for key in movie_features])
    except:
        print('failed on: {}'.format(inf['original_title']))

movie_df = pd.DataFrame(movie_records, columns=movie_features)

In [ ]:
# So this doesn't include the movieid, so that'll be important to incldue zip in the id too

cast_features = ['gender', 'id', 'name', 'order']

cast_records = []
for m_id, cast in zip(list(all_movies.id), credit_list):
    # Sometimes you get some crap status codes, I could handle this better but meh
    try: 
        for char in cast['cast']:
            tmp_list = []
            cst_list = [char.get(key) for key in cast_features]
            tmp_list.extend(cst_list)
            tmp_list.append(m_id)
            cast_records.append(tmp_list)
    except:
        pass
    
cast_df = pd.DataFrame(cast_records, columns=cast_features+['m_id'])

In [ ]:
# So this doesn't include the movieid, so that'll be important to incldue zip in the id too

cast_features = ['gender', 'id', 'name', 'order']

cast_records = []
for m_id, cast in zip(list(all_movies.id), credit_list):
    # Sometimes you get some crap status codes, I could handle this better but meh
    try: 
        for char in cast['cast']:
            tmp_list = []
            cst_list = [char.get(key) for key in cast_features]
            tmp_list.extend(cst_list)
            tmp_list.append(m_id)
            cast_records.append(tmp_list)
    except:
        pass
    
cast_df = pd.DataFrame(cast_records, columns=cast_features+['m_id'])

crew_features = ['gender', 'department', 'id', 'job', 'name']

crew_records = []
for m_id, cast in zip(list(all_movies.id), credit_list):
    # Sometimes you get some crap status codes, I could handle this better but meh
    try: 
        for char in cast['crew']:
            tmp_list = []
            crw_list = [char.get(key) for key in crew_features]
            tmp_list.extend(crw_list)
            tmp_list.append(m_id)
            crew_records.append(tmp_list)
    except:
        pass
    
crew_df = pd.DataFrame(crew_records, columns=crew_features+['m_id'])

In [84]:
cast_df.to_csv('cast_df.csv')
crew_df.to_csv('crew_df.csv')
movie_df.to_csv('movie_df.csv')